In [1]:
import os
import sys
import numpy as np
import sympy as sp
import torch as pt
from torch.nn.functional import normalize
import scipy
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split
import logging
import sys
from torch.utils.data import TensorDataset, DataLoader
from torch.func import hessian, vmap
from torch.optim.lr_scheduler import ReduceLROnPlateau

device = pt.device("cuda" if pt.cuda.is_available() else "cpu")

sys.path.append(os.path.abspath('../'))

from src.useful_functions import *
from src.PWDs_module import generate_PWDistances_torch
from src.sqra_functions import*
from src.openmm_functions import*
# For reproducibility
np.random.seed(0)


# Read directory paths
read_dirs_paths('dir_paths_.txt', globals())



 
Created variables:
inp_dir = /scratch/htc/fsafarov/structures/8ef5_july_2025/8ef5/
dcd_dir = /scratch/htc/fsafarov/mOR_dcd_files/npat/
out_dir = /scratch/htc/fsafarov/ISOKANN_PINN/output/


In [3]:
psf_file = 'step5_assembly.psf'
crd_file = 'step5_assembly.crd'

system, psf, crd = setup_system(
                    inp_dir,
                    ligand_name = '7V7',
                    nbcutoff = 1.0,
                    psf_file=psf_file,
                    crd_file=crd_file  
                )


dcd_file = 'mOR_simulation_NPAT_CA_aligned.dcd'
dt=2.0
gamma=1.0
T=310.15
platform='CUDA'


forces = get_parameters(
                        system,
                        inp_dir,
                        dcd_file,
                        dcd_dir,
                        dt, #in femtoseconds
                        T,
                        gamma,
                        platform,
                        pdb_file='step5_assembly.pdb',
                        integrator_type='Langevin',
                        psf=psf,
                        get_potential_grad=True
                        
                      )


Number of atoms in the system: 118294


/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/openmm/app/charmmparameterset.py:391: UserWarning: Replacing dihedral <DihedralType: k=0.27; phase=180.0; per=2> with <DihedralType: k=0.03; phase=180.0; per=2>
  warnings.warn('Replacing dihedral %r with %r' %
/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/openmm/app/charmmparameterset.py:391: UserWarning: Replacing dihedral <DihedralType: k=0.0; phase=0.0; per=3> with <DihedralType: k=0.12; phase=0.0; per=3>
  warnings.warn('Replacing dihedral %r with %r' %


dcdplugin) detected standard 32-bit DCD file of native endianness
dcdplugin) CHARMM format DCD file (also NAMD 2.1 and later)


In [4]:
coords = get_parameters(
                        system,
                        inp_dir,
                        dcd_file,
                        dcd_dir,
                        dt, #in femtoseconds
                        T,
                        gamma,
                        platform,
                        pdb_file='step5_assembly.pdb',
                        integrator_type='Langevin',
                        psf=psf,
                        get_coords=True
                      )

dcdplugin) detected standard 32-bit DCD file of native endianness
dcdplugin) CHARMM format DCD file (also NAMD 2.1 and later)


In [13]:
print(forces.shape)
print(coords.shape)

(5534, 118294, 3)
(5534, 118294, 3)


In [10]:
forces_norm = np.linalg.norm(forces, axis=-1)

In [12]:
forces_norm.shape

(5534, 118294)

In [8]:
positions = np.linalg.norm(coords, axis=-1)

In [9]:
positions.shape

(5534, 118294)